In [1]:
import pandas
import numpy 

import stock_data

# Background

#### High-level objectives:

1) Build daily table

2) Use daily table to build monthly table

#### How we will do this:

We are going to take the data yielded to us by the get_all_closing_prices() function and rework it a bit to (hopefully) make it more useful.. or at the very least, more interesting!

-- 

We will maintain the exact closing price values in our daily table - and will build off of this to calculate a basic returns column, a time delayed returns column, and a couple of other simple columns.

--

Some of these columns will involve a bit more calculation and/or data engineering. For these, there is a pretty standard template we will follow:

First, create an empty list. Then, loop through our 500 stock tickers. For each:

    1) Calculate values for new column
    2) Concatenate new column name to values
    3) Concatenate ticker (this will allow for neat alignment with existing multi-level column structure)
    4) Append this new DataFrame to our list

We will then join all of these together to create one DataFrame of these new values, with one column for each of our 500 tickers. 

--

Finally, we will perform two final joins to link all of these DataFrames together into our daily & monthly tables. 

# Get Data

In [2]:
data = stock_data.get_all_closing_prices("../stock_csvs/")

# Daily table

### Get closing price

#### Price

In [3]:
closing_prices = data.ix[:, data.columns.get_level_values(1).isin({"close"})]

#### Return 

In [4]:
closing_returns = (closing_prices.shift(-1) - closing_prices) / closing_prices

#### Previous day's return

In [5]:
previous_returns = closing_returns.shift(1)

### Get intraday values

#### Open to close

In [6]:
dataframes = []
for tic in closing_returns.columns.get_level_values(0):
    dataframes.append(
        pandas.concat({tic: 
            pandas.concat({'open_to_close':
                          (data[tic].close - data[tic].open)
                              / data[tic].open
                          }, axis=1)}, axis=1))

In [7]:
open_close = dataframes[0].join(dataframes[1:])

#### Span

In [8]:
dataframes = []
for tic in closing_returns.columns.get_level_values(0):
    dataframes.append(
        pandas.concat({tic: 
            pandas.concat({'intra_span':
                          (data[tic].high - data[tic].low_price)
                              / data[tic].low_price
                          }, axis=1)}, axis=1))

In [9]:
intra_span = dataframes[0].join(dataframes[1:])

### Get volume

#### Day's Volume

In [10]:
volume =  data.ix[:, data.columns.get_level_values(1).isin({"volume"})]

#### % above previous day's volume

In [11]:
volume_returns =  (volume.shift(-1) - volume) / volume

### Get return of most-correlated other stock

In [12]:
corrs = stock_data.get_max_corrs(closing_returns)

In [13]:
dataframes = []
for tic in closing_returns.columns.get_level_values(0):
    top_corr = corrs.ix[tic, 'most correlated other stock']
    dataframes.append(
        pandas.concat({tic: 
            pandas.concat({'corr_other': 
                          closing_returns[top_corr].close}, axis=1)}, axis=1))

In [14]:
all_corrs = dataframes[0].join(dataframes[1:])

### Combine

In [15]:
dataframes = []
for tic in closing_returns.columns.get_level_values(0):
    
    dataframes.append(pandas.concat({
                tic: closing_prices[tic].join(
                        closing_returns[tic], rsuffix='_return').join(
                            previous_returns[tic], rsuffix='_return_previous').join(
                                open_close[tic]).join(
                                    intra_span[tic]).join(
                                        volume[tic]).join(
                                              volume_returns[tic], rsuffix='_change').join(
                                                    all_corrs[tic])
                }, axis=1))

In [16]:
daily_table = dataframes[0].join(dataframes[1:])

In [17]:
daily_table.aa.head()

,close,close_return,close_return_previous,open_to_close,intra_span,volume,volume_change,corr_other
date,,,,,,,,
1998-01-02,13.3511,0.017542,NaN,-0.001817,0.013294,2739839.602,0.946435,-0.006130
1998-01-05,13.5853,-0.022348,0.017542,0.006945,0.017388,5332919.264,-0.246372,-0.012006
1998-01-06,13.2817,0.001694,-0.022348,-0.017328,0.017633,4019035.876,-0.141415,0.026607
1998-01-07,13.3042,-0.041408,0.001694,0.020699,0.020699,3450682.202,-0.292607,0.000000
1998-01-08,12.7533,-0.023806,-0.041408,-0.041408,0.051467,2440987.524,2.110187,-0.080000


# Monthly table

### Create initial table

#### Average across months in daily table

In [18]:
monthly_table = daily_table.groupby([daily_table.index.year, daily_table.index.month]).mean()

In [19]:
monthly_table.aa.head()

close  close_return  close_return_previous  open_to_close  \
1998 1  13.001770      0.004727               0.003860       0.002060   
     2  14.031084     -0.004858              -0.001855      -0.001103   
     3  13.463482     -0.001372              -0.002711      -0.003541   
     4  13.977748      0.006396               0.005821       0.000265   
     5  13.955640     -0.006089              -0.005222      -0.006265   

        intra_span        volume  volume_change  corr_other  
1998 1    0.023501  5.295351e+06       0.166955    0.007549  
     2    0.018260  3.665675e+06       0.165154    0.001046  
     3    0.020624  4.244956e+06       0.064034    0.002662  
     4    0.020576  6.629143e+06       0.056076    0.001971  
     5    0.016760  3.316375e+06       0.148095   -0.005262

### Get monthly return

#### Calulate monthly return for each stock

In [20]:
dataframes = []
for tic in closing_returns.columns.get_level_values(0):
    dataframes.append(
        pandas.concat({tic: 
            pandas.concat({'return_monthly':
                (daily_table[tic].close_return + 1).groupby([daily_table.index.year, 
                                                             daily_table.index.month]).apply(lambda x: 
                                                                                        numpy.product(x)).groupby(
                                                                                [daily_table.index.year, 
                                                                                 daily_table.index.month]).first()
                          }, axis=1)}, axis=1))

#### Join results into one DataFrame

In [21]:
monthly_returns = dataframes[0].join(dataframes[1:])

#### Check to see if this works

In [22]:
((daily_table.aa.close[0] * 
    monthly_returns.aa.return_monthly[0]).round(4) == 
        daily_table.aa.close[(daily_table.index.year==1998) & (daily_table.index.month==2)][0])

True

#### Notes:

To check our monthly compunded return calculations, what we are doing is:

    1) Pick an example ticker (in this case 'aa', as it has complete data)
    2) Grab the monthly compounded return value for the month of January in 1998
    3) Multiply it by the first closing price we have for 'aa' (Jan 2nd, 1998) and round to 4 decimal places
    4) And compare that to the first closing price we have for February of 1998

### Get variance of daily returns

In [23]:
dataframes = []
for tic in closing_returns.columns.get_level_values(0):
    dataframes.append(
        pandas.concat({tic: 
            pandas.concat({'close_return_var':
                daily_table[tic].close_return.groupby([daily_table.index.year, 
                                                       daily_table.index.month]).apply(lambda x: 
                                                                                           numpy.var(x))
                          }, axis=1)}, axis=1))

In [24]:
monthly_ret_var = dataframes[0].join(dataframes[1:])

### Monthly high, as a % of monthly average close

In [25]:
dataframes = []
for tic in closing_returns.columns.get_level_values(0):
    dataframes.append(
        pandas.concat({tic: 
            pandas.concat({'high_monthly':
                daily_table[tic].close.groupby([daily_table.index.year, 
                                                daily_table.index.month]).max().divide(
                                                        monthly_table[tic].close)
                          }, axis=1)}, axis=1))

In [26]:
monthly_high = (dataframes[0].join(dataframes[1:]))

### Monthly low, as a % of monthly average close

In [27]:
dataframes = []
for tic in closing_returns.columns.get_level_values(0):
    dataframes.append(
        pandas.concat({tic: 
            pandas.concat({'low_monthly':
                daily_table[tic].close.groupby([daily_table.index.year, 
                                                daily_table.index.month]).min().divide(
                                                        monthly_table[tic].close)
                          }, axis=1)}, axis=1))

In [28]:
monthly_low = (dataframes[0].join(dataframes[1:]))

### Combine

In [29]:
dataframes = []
for tic in closing_returns.columns.get_level_values(0):
    
    dataframes.append(pandas.concat({
                tic: monthly_table[tic].join(
                        monthly_returns[tic]).join(
                            monthly_ret_var[tic]).join(
                                monthly_high[tic]).join(
                                    monthly_low[tic])
                }, axis=1))

In [30]:
monthly_table = dataframes[0].join(dataframes[1:])

In [31]:
monthly_table.aa.head()

close  close_return  close_return_previous  open_to_close  \
1998 1  13.001770      0.004727               0.003860       0.002060   
     2  14.031084     -0.004858              -0.001855      -0.001103   
     3  13.463482     -0.001372              -0.002711      -0.003541   
     4  13.977748      0.006396               0.005821       0.000265   
     5  13.955640     -0.006089              -0.005222      -0.006265   

        intra_span        volume  volume_change  corr_other  return_monthly  \
1998 1    0.023501  5.295351e+06       0.166955    0.007549        1.094734   
     2    0.018260  3.665675e+06       0.165154    0.001046        0.910002   
     3    0.020624  4.244956e+06       0.064034    0.002662        0.967347   
     4    0.020576  6.629143e+06       0.056076    0.001971        1.138813   
     5    0.016760  3.316375e+06       0.148095   -0.005262        0.884181   

        close_return_var  high_monthly  low_monthly  
1998 1          0.000383      1.100796     0.949471  
     2          0.000184      1.041680     0.947931  
     3          0.000272      1.037607     0.952993  
     4          0.000382      1.061702     0.911198  
     5          0.000094      1.052471     0.939255

# And that's it

#### Notes:

There are many other features we can build, but we have at least enough to get us started with a little modeling. 

# Output

In [32]:
daily_table.to_pickle("../output/daily_table.pkl")

In [33]:
monthly_table.to_pickle("../output/monthly_table.pkl")

#### Notes:

Ouputting to .pkl format will work better than .csv for a large dataset like this. 